## <center>Курсовой проект<a class="anchor" id="course_project"></a><center>

Ссылка на соревнование - https://www.kaggle.com/c/geekbrains-competitive-data-analysis/data

Постановка задачи.

Для построения модели в данном соревновании, сначала нужно будет собрать выборку для обучения модели. Формат соревнования очень похож на то, как в промышленности Data Scinetist'ы строят алгоритмы: сначала нужно провести анализ данных, собрать выборку и после этого строить модели. В соревновании представлены 4 типы источника данных, которые могут быть интерпретированы как таблицы в базе данных. Некоторые источники данных уже готовы для моделирования, представлены в агрерированном виде. Другие источники данных требуется представить в удобном для модели виде.

Описание источников данных:

    train.csv - пары "заявка - целевая переменная", для этой выборки нужно собрать признаки и обучить модель;
    test.csv - пары "заявки - прогнозное значение", для этой выборки нужно собрать признаки и построить прогнозы;
    bki.csv - данные БКИ о предыдущих кредитах клиента;
    client_profile.csv - клиентский профиль, некоторые знания, которые есть у компании о клиенте;
    payments.csv - история платежей клиента;
    applications_history.csv - история предыдущих заявок клиента.


In [79]:
import re
import numpy as np
import pandas as pd
import xgboost as xgb
import catboost as cb
import lightgbm as lgb
import matplotlib.pyplot as plt
import missingno as msno
from sklearn import metrics
from datetime import timedelta
from datetime import datetime
import xgboost as xgb, lightgbm as lgbm, catboost as catb

from sklearn.model_selection import train_test_split
# from sunbird.categorical_encoding import frequency_encoding
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import seaborn as sns
import scipy.stats as st

import warnings
warnings.simplefilter("ignore")

import warnings
from tqdm import tqdm
from typing import List, Tuple

from scipy.stats import ttest_rel

# from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score

warnings.simplefilter("ignore")
%matplotlib inline

In [80]:
train_df = pd.read_csv('../data/geekbrains-competitive-data-analysis/train.csv')
train_df.head(4)

APPLICATION_NUMBER  TARGET NAME_CONTRACT_TYPE
0           123687442       0               Cash
1           123597908       1               Cash
2           123526683       0               Cash
3           123710391       1               Cash

In [81]:
app_history_df = pd.read_csv('../data/geekbrains-competitive-data-analysis/applications_history.csv')
app_history_df.head(4)

PREV_APPLICATION_NUMBER  APPLICATION_NUMBER NAME_CONTRACT_TYPE  \
0                 49298709           123595216                NaN   
1                 50070639           123431468               Cash   
2                 49791680           123445379               Cash   
3                 50087457           123499497               Cash   

   AMOUNT_ANNUITY  AMT_APPLICATION  AMOUNT_CREDIT  AMOUNT_PAYMENT  \
0        1730.430          17145.0        17145.0             0.0   
1       25188.615         607500.0       679671.0             NaN   
2       15060.735         112500.0       136444.5             NaN   
3       47041.335         450000.0       470790.0             NaN   

   AMOUNT_GOODS_PAYMENT NAME_CONTRACT_STATUS  DAYS_DECISION  ...  \
0               17145.0             Approved             73  ...   
1              607500.0             Approved            164  ...   
2              112500.0             Approved            301  ...   
3              450000.0             Approved            512  ...   

  NAME_PRODUCT_TYPE SELLERPLACE_AREA CNT_PAYMENT NAME_YIELD_GROUP  \
0               XNA               35        12.0           middle   
1            x-sell               -1        36.0       low_action   
2            x-sell               -1        12.0             high   
3            x-sell               -1        12.0           middle   

  DAYS_FIRST_DRAWING DAYS_FIRST_DUE DAYS_LAST_DUE_1ST_VERSION  DAYS_LAST_DUE  \
0           365243.0           42.0                     300.0           42.0   
1           365243.0          134.0                     916.0       365243.0   
2           365243.0          271.0                      59.0       365243.0   
3           365243.0          482.0                     152.0          182.0   

   DAYS_TERMINATION NFLAG_INSURED_ON_APPROVAL  
0              37.0                       0.0  
1          365243.0                       1.0  
2          365243.0                       1.0  
3             177.0                       1.0  

[4 rows x 26 columns]

In [82]:
bki_df = pd.read_csv('../data/geekbrains-competitive-data-analysis/bki.csv')
bki_df.head(4)

APPLICATION_NUMBER  BUREAU_ID CREDIT_ACTIVE CREDIT_CURRENCY  DAYS_CREDIT  \
0           123538884    5223613        Active      currency 1        718.0   
1           123436670    6207544        Closed      currency 1        696.0   
2           123589020    6326395        Closed      currency 1        165.0   
3           123494590    6606618        Active      currency 1         55.0   

   CREDIT_DAY_OVERDUE  DAYS_CREDIT_ENDDATE  DAYS_ENDDATE_FACT  \
0                   0                377.0                NaN   
1                   0                511.0              511.0   
2                   0                149.0              160.0   
3                   0                310.0                NaN   

   AMT_CREDIT_MAX_OVERDUE  CNT_CREDIT_PROLONG  AMT_CREDIT_SUM  \
0                19386.81                   0       675000.00   
1                    0.00                   0        93111.66   
2                     NaN                   0        36000.00   
3                     NaN                   0        38664.00   

   AMT_CREDIT_SUM_DEBT  AMT_CREDIT_SUM_LIMIT  AMT_CREDIT_SUM_OVERDUE  \
0           320265.495                   0.0                     0.0   
1                0.000                   0.0                     0.0   
2                0.000                   0.0                     0.0   
3            37858.500                   NaN                     0.0   

       CREDIT_TYPE  DAYS_CREDIT_UPDATE  AMT_ANNUITY  
0  Consumer credit                39.0          NaN  
1  Consumer credit               505.0          NaN  
2  Consumer credit               150.0          0.0  
3  Consumer credit                15.0          NaN

In [83]:
client_profile_df = pd.read_csv('../data/geekbrains-competitive-data-analysis/client_profile.csv')
client_profile_df.head(4)

APPLICATION_NUMBER GENDER  CHILDRENS  TOTAL_SALARY  AMOUNT_CREDIT  \
0           123666076      F          0      157500.0       270000.0   
1           123423688      F          0      270000.0       536917.5   
2           123501780      M          1      427500.0       239850.0   
3           123588799      M          0      112500.0       254700.0   

   AMOUNT_ANNUITY                EDUCATION_LEVEL   FAMILY_STATUS  \
0         13500.0              Incomplete higher  Civil marriage   
1         28467.0  Secondary / secondary special         Married   
2         23850.0              Incomplete higher         Married   
3         17149.5  Secondary / secondary special         Married   

   REGION_POPULATION    AGE  ...  FAMILY_SIZE  EXTERNAL_SCORING_RATING_1  \
0           0.008068   8560  ...          2.0                   0.329471   
1           0.020246  23187  ...          2.0                        NaN   
2           0.072508  14387  ...          3.0                   0.409017   
3           0.019101  14273  ...          2.0                        NaN   

   EXTERNAL_SCORING_RATING_2  EXTERNAL_SCORING_RATING_3  \
0                   0.236315                   0.678568   
1                   0.442295                   0.802745   
2                   0.738159                        NaN   
3                   0.308994                   0.590233   

   AMT_REQ_CREDIT_BUREAU_HOUR  AMT_REQ_CREDIT_BUREAU_DAY  \
0                         0.0                        0.0   
1                         0.0                        0.0   
2                         NaN                        NaN   
3                         0.0                        0.0   

   AMT_REQ_CREDIT_BUREAU_WEEK  AMT_REQ_CREDIT_BUREAU_MON  \
0                         0.0                        0.0   
1                         0.0                        0.0   
2                         NaN                        NaN   
3                         0.0                        0.0   

   AMT_REQ_CREDIT_BUREAU_QRT  AMT_REQ_CREDIT_BUREAU_YEAR  
0                        1.0                         2.0  
1                        1.0                         1.0  
2                        NaN                         NaN  
3                        0.0                         3.0  

[4 rows x 24 columns]

In [84]:
payments_df = pd.read_csv('../data/geekbrains-competitive-data-analysis/payments.csv')
payments_df.head(4)

PREV_APPLICATION_NUMBER  APPLICATION_NUMBER  NUM_INSTALMENT_VERSION  \
0                 49011181           123664960                     1.0   
1                 48683432           123497205                     1.0   
2                 48652024           123749925                     1.0   
3                 48398897           123550846                     0.0   

   NUM_INSTALMENT_NUMBER  DAYS_INSTALMENT  DAYS_ENTRY_PAYMENT  AMT_INSTALMENT  \
0                      5           1002.0              1015.0       12156.615   
1                     13            442.0               432.0       18392.535   
2                     10              8.0                23.0        5499.945   
3                     82            398.0               398.0        7082.145   

   AMT_PAYMENT  
0    12156.615  
1    10047.645  
2     5499.945  
3     7082.145

In [85]:
payments_df['APPLICATION_NUMBER'].value_counts()

123619703    37
123486520    33
123449869    31
123682859    31
123632163    31
             ..
123731958     1
123778085     1
123722215     1
123639711     1
123560468     1
Name: APPLICATION_NUMBER, Length: 264726, dtype: int64

Проведем анализ признаков. 

Постараемся собрать из несколькоих таблиц одну.

посмотрим что есть по по клиентам. 

посмотреть сколько кредитов в среднем у человека есть. 

сколько кредитов погашено, а сколько осталось.




In [86]:
app_history_df.loc[((app_history_df['NAME_CONTRACT_STATUS']=='Approved') & app_history_df['NFLAG_INSURED_ON_APPROVAL'].isna())]

PREV_APPLICATION_NUMBER  APPLICATION_NUMBER NAME_CONTRACT_TYPE  \
18                      49661323           123719644               Cash   
47                      48743952           123559076               Cash   
74                      48653311           123692762                NaN   
172                     48650361           123659969               Cash   
239                     49488458           123626811                NaN   
...                          ...                 ...                ...   
1670146                 48882557           123639067               Cash   
1670150                 48825150           123493032               Cash   
1670159                 49622726           123716694               Cash   
1670175                 48444623           123571293               Cash   
1670177                 49755203           123714856               Cash   

         AMOUNT_ANNUITY  AMT_APPLICATION  AMOUNT_CREDIT  AMOUNT_PAYMENT  \
18            10181.700         180000.0       180000.0             NaN   
47            75897.630        2025000.0      2241513.0             NaN   
74             2652.705          19611.0        14170.5          6111.0   
172           10061.910         112500.0       127350.0             NaN   
239           41383.710         234243.0       234243.0             0.0   
...                 ...              ...            ...             ...   
1670146       83758.275        2025000.0      2169342.0             NaN   
1670150       18852.435          90000.0        92970.0             NaN   
1670159       20380.500         450000.0       450000.0             NaN   
1670175        9740.250         135000.0       135000.0             NaN   
1670177       20876.805         270000.0       299223.0             NaN   

         AMOUNT_GOODS_PAYMENT NAME_CONTRACT_STATUS  DAYS_DECISION  ...  \
18                   180000.0             Approved            700  ...   
47                  2025000.0             Approved            524  ...   
74                    19611.0             Approved           1525  ...   
172                  112500.0             Approved            631  ...   
239                  234243.0             Approved            269  ...   
...                       ...                  ...            ...  ...   
1670146             2025000.0             Approved            678  ...   
1670150               90000.0             Approved            253  ...   
1670159              450000.0             Approved            706  ...   
1670175              135000.0             Approved           1010  ...   
1670177              270000.0             Approved            869  ...   

        NAME_PRODUCT_TYPE SELLERPLACE_AREA CNT_PAYMENT NAME_YIELD_GROUP  \
18                 x-sell                6        24.0       low_normal   
47                 x-sell               -1        54.0       low_normal   
74                    XNA             2078         6.0           middle   
172               walk-in                4        24.0             high   
239                   XNA              100         6.0       low_normal   
...                   ...              ...         ...              ...   
1670146            x-sell               -1        36.0       low_normal   
1670150            x-sell               -1         6.0             high   
1670159            x-sell               -1        48.0           middle   
1670175            x-sell               -1        18.0           middle   
1670177            x-sell               -1        24.0             high   

        DAYS_FIRST_DRAWING DAYS_FIRST_DUE DAYS_LAST_DUE_1ST_VERSION  \
18                     NaN            NaN                       NaN   
47                     NaN            NaN                       NaN   
74                     NaN            NaN                       NaN   
172                    NaN            NaN                       NaN   
239                    NaN            NaN                     

In [87]:
app_history_df['NFLAG_INSURED_ON_APPROVAL'].isna()

0          False
1          False
2          False
3          False
4           True
           ...  
1670209    False
1670210    False
1670211    False
1670212    False
1670213    False
Name: NFLAG_INSURED_ON_APPROVAL, Length: 1670214, dtype: bool

In [88]:
# Проведем анализ таблицы БКИ 

# Попробуем выявить зависимости.  

# Узнаем что за собой скрывают пропуски.

In [89]:
bki_df['CREDIT_CURRENCY'].value_counts()

currency 1    944492
currency 2       647
currency 3        91
currency 4         4
Name: CREDIT_CURRENCY, dtype: int64

In [90]:
bki_df['CREDIT_ACTIVE'].value_counts()

Closed      594315
Active      347323
Sold          3583
Bad debt        13
Name: CREDIT_ACTIVE, dtype: int64

In [91]:
# Объединим таблицы. 
train_df.merge(client_profile_df,how='inner', on='APPLICATION_NUMBER')

APPLICATION_NUMBER  TARGET NAME_CONTRACT_TYPE GENDER  CHILDRENS  \
0               123687442       0               Cash      M          1   
1               123526683       0               Cash      F          0   
2               123710391       1               Cash      M          0   
3               123718821       0               Cash      F          1   
4               123544624       0               Cash      M          0   
...                   ...     ...                ...    ...        ...   
89534           123595479       0               Cash      F          0   
89535           123426326       0               Cash      F          2   
89536           123672463       0               Cash      F          0   
89537           123554358       0               Cash      F          2   
89538           123676265       0        Credit Card      F          1   

       TOTAL_SALARY  AMOUNT_CREDIT  AMOUNT_ANNUITY  \
0          157500.0       855000.0         25128.0   
1          135000.0      1006920.0         42660.0   
2          180000.0       518562.0         22972.5   
3          180000.0       755190.0         36459.0   
4          202500.0      1078200.0         38331.0   
...             ...            ...             ...   
89534      225000.0       990432.0         32863.5   
89535      135000.0       995643.0         32904.0   
89536      175500.0       269550.0         12618.0   
89537      270000.0      1024740.0         49428.0   
89538      112500.0       337500.0         16875.0   

                     EDUCATION_LEVEL FAMILY_STATUS  ...  FAMILY_SIZE  \
0      Secondary / secondary special       Married  ...          3.0   
1                   Higher education       Married  ...          2.0   
2      Secondary / secondary special       Married  ...          2.0   
3      Secondary / secondary special       Married  ...          3.0   
4      Secondary / secondary special       Married  ...          2.0   
...                              ...           ...  ...          ...   
89534  Secondary / secondary special     Separated  ...          1.0   
89535  Secondary / secondary special       Married  ...          4.0   
89536  Secondary / secondary special       Married  ...          2.0   
89537              Incomplete higher       Married  ...          4.0   
89538  Secondary / secondary special       Married  ...          3.0   

       EXTERNAL_SCORING_RATING_1  EXTERNAL_SCORING_RATING_2  \
0                       0.700784                   0.645914   
1                            NaN                   0.682149   
2                            NaN                   0.171299   
3                            NaN                   0.548504   
4                            NaN                   0.547421   
...                          ...                        ...   
89534                   0.768198                   0.713399   
89535                   0.410920                   0.592042   
89536                   0.528096                   0.471774   
89537                   0.653826                   0.651227   
89538                        NaN                   0.552043   

       EXTERNAL_SCORING_RATING_3  AMT_REQ_CREDIT_BUREAU_HOUR  \
0                       0.716570                         0.0   
1                       0.267869                         0.0   
2                       0.170446                         0.0   
3                       0.331251                         1.0   
4                       0.740799                         0.0   
...                          ...                         ...   
89534                   0.529890                         0.0   
89535                   0.570917                         0.0   
89536                   0.581484                         0.0   
89537                   0.558507                         0.0   
89538                        NaN                         NaN   

       AMT_REQ_CREDIT_BUREAU_DAY  AMT_REQ_CREDIT_BUREAU_WEEK  \
0             

In [92]:
client_profile_df.loc[(client_profile_df['APPLICATION_NUMBER']==123590329)]

Empty DataFrame
Columns: [APPLICATION_NUMBER, GENDER, CHILDRENS, TOTAL_SALARY, AMOUNT_CREDIT, AMOUNT_ANNUITY, EDUCATION_LEVEL, FAMILY_STATUS, REGION_POPULATION, AGE, DAYS_ON_LAST_JOB, OWN_CAR_AGE, FLAG_PHONE, FLAG_EMAIL, FAMILY_SIZE, EXTERNAL_SCORING_RATING_1, EXTERNAL_SCORING_RATING_2, EXTERNAL_SCORING_RATING_3, AMT_REQ_CREDIT_BUREAU_HOUR, AMT_REQ_CREDIT_BUREAU_DAY, AMT_REQ_CREDIT_BUREAU_WEEK, AMT_REQ_CREDIT_BUREAU_MON, AMT_REQ_CREDIT_BUREAU_QRT, AMT_REQ_CREDIT_BUREAU_YEAR]
Index: []

[0 rows x 24 columns]

In [93]:
 bki_df.loc[(bki_df['APPLICATION_NUMBER']==123723001)]

APPLICATION_NUMBER  BUREAU_ID CREDIT_ACTIVE CREDIT_CURRENCY  \
762838           123723001    6177264        Closed      currency 1   
815649           123723001    6087688        Closed      currency 1   
857472           123723001    6138700        Closed      currency 1   

        DAYS_CREDIT  CREDIT_DAY_OVERDUE  DAYS_CREDIT_ENDDATE  \
762838       1945.0                   0               1641.0   
815649       1946.0                   0               1641.0   
857472       1551.0                   0               1248.0   

        DAYS_ENDDATE_FACT  AMT_CREDIT_MAX_OVERDUE  CNT_CREDIT_PROLONG  \
762838             1641.0                     NaN                   0   
815649             1641.0                     0.0                   0   
857472             1125.0                     NaN                   0   

        AMT_CREDIT_SUM  AMT_CREDIT_SUM_DEBT  AMT_CREDIT_SUM_LIMIT  \
762838         43740.0                  0.0                   0.0   
815649         43740.0                  0.0                   0.0   
857472         76810.5                  NaN                   NaN   

        AMT_CREDIT_SUM_OVERDUE      CREDIT_TYPE  DAYS_CREDIT_UPDATE  \
762838                     0.0  Consumer credit               875.0   
815649                     0.0  Consumer credit               881.0   
857472                     0.0  Consumer credit              1122.0   

        AMT_ANNUITY  
762838          0.0  
815649          0.0  
857472          0.0

In [94]:
app_history_df.loc[(app_history_df['APPLICATION_NUMBER']==123723001)]

PREV_APPLICATION_NUMBER  APPLICATION_NUMBER NAME_CONTRACT_TYPE  \
915251                 49650758           123723001                NaN   
962266                 49065356           123723001                NaN   

        AMOUNT_ANNUITY  AMT_APPLICATION  AMOUNT_CREDIT  AMOUNT_PAYMENT  \
915251        4998.645          34245.0        27207.0          8563.5   
962266        3001.950          26482.5        29281.5             0.0   

        AMOUNT_GOODS_PAYMENT NAME_CONTRACT_STATUS  DAYS_DECISION  ...  \
915251               34245.0             Approved            190  ...   
962266               26482.5             Approved            739  ...   

       NAME_PRODUCT_TYPE SELLERPLACE_AREA CNT_PAYMENT NAME_YIELD_GROUP  \
915251               XNA              156         6.0       low_normal   
962266               XNA              156        12.0           middle   

       DAYS_FIRST_DRAWING DAYS_FIRST_DUE DAYS_LAST_DUE_1ST_VERSION  \
915251           365243.0          160.0                      10.0   
962266           365243.0          708.0                     378.0   

        DAYS_LAST_DUE  DAYS_TERMINATION NFLAG_INSURED_ON_APPROVAL  
915251           40.0              34.0                       1.0  
962266          558.0             526.0                       0.0  

[2 rows x 26 columns]

In [95]:
payments_df.loc[(payments_df['APPLICATION_NUMBER']==123723001)]

PREV_APPLICATION_NUMBER  APPLICATION_NUMBER  NUM_INSTALMENT_VERSION  \
752132                 49650758           123723001                     1.0   

        NUM_INSTALMENT_NUMBER  DAYS_INSTALMENT  DAYS_ENTRY_PAYMENT  \
752132                      4             70.0                83.0   

        AMT_INSTALMENT  AMT_PAYMENT  
752132        4998.645     4998.645

In [96]:
bki_df['APPLICATION_NUMBER'].value_counts()

123444199    63
123493043    51
123641404    50
123603494    39
123574982    36
             ..
123503595     1
123530515     1
123664592     1
123775289     1
123685733     1
Name: APPLICATION_NUMBER, Length: 273131, dtype: int64

In [97]:
payments_df['APPLICATION_NUMBER'].value_counts()#==123723001)]

123619703    37
123486520    33
123449869    31
123682859    31
123632163    31
             ..
123731958     1
123778085     1
123722215     1
123639711     1
123560468     1
Name: APPLICATION_NUMBER, Length: 264726, dtype: int64

In [98]:
bki_df['CREDIT_ACTIVE'].value_counts()

Closed      594315
Active      347323
Sold          3583
Bad debt        13
Name: CREDIT_ACTIVE, dtype: int64

In [99]:
credit_curency = {'currency 1':0, 'currency 2':1, 'currency 3':2, 'currency 4':3}

In [100]:
bki_df['CREDIT_CURRENCY'] = bki_df['CREDIT_CURRENCY'].apply(lambda x: credit_curency[x])

In [101]:
credit_type = bki_df['CREDIT_TYPE'].value_counts()

In [102]:
bki_dic = {
    'APPLICATION_NUMBER':[],
     'active_credits':[],
     'closed_credits':[],
     'sold_credits':[],
     'bad_debt':[],
     'credit_problems':[], 
     'credits_count':[],
     'curency_mean':[]
}
 

In [103]:
def creat_features(app_number):
    bki = bki_df.loc[(bki_df['APPLICATION_NUMBER']==app_number)]

    credit_problems = np.sum(bki['CNT_CREDIT_PROLONG'])

    active_credits = 0
    closed_credits = 0
    sold_credits = 0
    bad_debt = 0

    for status in bki['CREDIT_ACTIVE']:
        if status == 'Active':
            active_credits +=1
        if status == 'Closed':
            closed_credits +=1
        if status == 'Sold':
            sold_credits +=1
        if status == 'Bad debt':
            bad_debt +=1  

    active_credits, closed_credits, sold_credits, bad_debt  
    credits_count = len(bki)
    curency_mean = np.mean(bki['CREDIT_CURRENCY'])
    bki_dic['APPLICATION_NUMBER'].append(app_number)
    bki_dic['active_credits'].append(active_credits)
    bki_dic['closed_credits'].append(closed_credits)
    bki_dic['sold_credits'].append(sold_credits)
    bki_dic['bad_debt'].append(bad_debt)
    bki_dic['credit_problems'].append(credit_problems)
    bki_dic['credits_count'].append(credits_count)
    bki_dic['curency_mean'].append(curency_mean)

In [ ]:
bki_df['APPLICATION_NUMBER'].unique()

In [158]:
bki_unic = bki_df['APPLICATION_NUMBER'].unique()

for app_number in tqdm(bki_unic):
    creat_features(app_number)

bki_statistic = pd.DataFrame(bki_dic)
bki_statistic.to_csv('bki_statistic.csv',index=False)

100%|██████████| 273131/273131 [50:09<00:00, 90.77it/s] 


In [159]:
np.max(bki_statistic['APPLICATION_NUMBER'].value_counts().values)

1

In [104]:
# bki_statistic = pd.read_csv('bki_statistic.csv')

In [160]:
bki_statistic.head(5)

APPLICATION_NUMBER  active_credits  closed_credits  sold_credits  bad_debt  \
0           123538884               2               1             0         0   
1           123436670               5               3             0         0   
2           123589020               0               7             0         0   
3           123494590               2               1             0         0   
4           123446603               1               0             0         0   

   credit_problems  credits_count  curency_mean  
0                0              3           0.0  
1                0              8           0.0  
2                0              7           0.0  
3                0              3           0.0  
4                0              1           0.0

In [161]:
bki_statistic['credit_problems'].value_counts()

0    268241
1      4036
2       683
3       121
4        36
5        11
6         2
7         1
Name: credit_problems, dtype: int64

In [162]:
train_df['TARGET'].value_counts()

0    101196
1      8897
Name: TARGET, dtype: int64

In [110]:
# Сгруппируем признаки из набора данных так чтобы получилось что-то адекватное.

In [163]:
categorical = []
numerical = []
for col, value in bki_df.iteritems():
    if value.dtype == 'object':
        categorical.append(col)
    else:
        numerical.append(col)

In [164]:
categorical

['CREDIT_ACTIVE', 'CREDIT_TYPE']

In [165]:
client_profile_df['FAMILY_STATUS'].value_counts()

Married                 159846
Single / not married     36883
Civil marriage           24252
Separated                15984
Widow                    13033
Unknown                      2
Name: FAMILY_STATUS, dtype: int64

In [166]:
client_profile_df['EDUCATION_LEVEL'].value_counts()

Secondary / secondary special    177562
Higher education                  60864
Incomplete higher                  8357
Lower secondary                    3081
Academic degree                     136
Name: EDUCATION_LEVEL, dtype: int64

In [167]:
client_profile_df.loc[(client_profile_df['GENDER']=='XNA')]

APPLICATION_NUMBER GENDER  CHILDRENS  TOTAL_SALARY  AMOUNT_CREDIT  \
23011            123506723    XNA          1      135000.0       405000.0   
34379            123458998    XNA          0      207000.0       382500.0   
81943            123461907    XNA          2      157500.0       270000.0   
178975           123612981    XNA          0      247500.0       540000.0   

        AMOUNT_ANNUITY                EDUCATION_LEVEL   FAMILY_STATUS  \
23011          20250.0               Higher education         Married   
34379          19125.0  Secondary / secondary special         Married   
81943          13500.0  Secondary / secondary special         Married   
178975         27000.0              Incomplete higher  Civil marriage   

        REGION_POPULATION    AGE  ...  FAMILY_SIZE  EXTERNAL_SCORING_RATING_1  \
23011            0.035792  10647  ...          3.0                   0.404970   
34379            0.020713  20232  ...          2.0                        NaN   
81943            0.026392  13717  ...          4.0                        NaN   
178975           0.035792   9649  ...          2.0                   0.653035   

        EXTERNAL_SCORING_RATING_2  EXTERNAL_SCORING_RATING_3  \
23011                    0.659185                   0.076984   
34379                    0.295998                   0.461482   
81943                    0.709205                   0.310818   
178975                   0.658620                   0.360613   

        AMT_REQ_CREDIT_BUREAU_HOUR  AMT_REQ_CREDIT_BUREAU_DAY  \
23011                          0.0                        0.0   
34379                          0.0                        0.0   
81943                          0.0                        0.0   
178975                         0.0                        0.0   

        AMT_REQ_CREDIT_BUREAU_WEEK  AMT_REQ_CREDIT_BUREAU_MON  \
23011                          0.0                        0.0   
34379                          0.0                        0.0   
81943                          0.0                        3.0   
178975                         0.0                        0.0   

        AMT_REQ_CREDIT_BUREAU_QRT  AMT_REQ_CREDIT_BUREAU_YEAR  
23011                         0.0                         3.0  
34379                         0.0                         1.0  
81943                         0.0                         4.0  
178975                        1.0                         6.0  

[4 rows x 24 columns]

In [168]:
def get_freq_encoder(data,feature_names):
    for feature_name in feature_names:
        freq_encoder = data[feature_name].value_counts(normalize=True)
        data[f"{feature_name}_freq_enc"] = data[feature_name].map(freq_encoder)
    return data

In [169]:
def category_to_digit(df, features):
    df = df.copy(deep=True)
    for i, feature in enumerate(features):
        # feature = str.replace(feature,' ','_')
        values_list = df[feature].value_counts()
        names = sorted(values_list.index)
        # names = sorted(names)
        for name in names:
            name = str.replace(name,' ','_')
            df.insert(3, f'{feature}_{name}', np.where((df[feature]==name),1,0), True)
    df.drop(features, axis=1, inplace=True)
    return df

In [179]:
def df_prepare(source_df, join_type ='left'):
    source_df = source_df.copy(deep=True)
    client_profile_df_1 = category_to_digit(client_profile_df, ['EDUCATION_LEVEL', 'FAMILY_STATUS'])
    df = source_df.merge(client_profile_df_1, how=join_type, on='APPLICATION_NUMBER')
    df = df.merge(bki_statistic, how=join_type ,on='APPLICATION_NUMBER')
    
    df['GENDER'] = np.where(df['GENDER']=='M', 1, 0)
    df['AGE'] = df['AGE'].apply(lambda x: x / 365)
    df['DAYS_ON_LAST_JOB'] = df['DAYS_ON_LAST_JOB'].apply(lambda x: x / 365)
    df = category_to_digit(df, ['NAME_CONTRACT_TYPE'])
    
    return df

In [180]:
categorical = []
numerical = []
for col, value in client_profile_df.iteritems():
    if value.dtype == 'object':
        categorical.append(col)
    else:
        numerical.append(col)
categorical

['GENDER', 'EDUCATION_LEVEL', 'FAMILY_STATUS']

In [120]:
# Попробуем обучить модель на этих данных.

In [181]:
def balance_df_by_target(df, target_name, method='over'):

    assert method in ['over', 'under', 'tomek', 'smote'], 'Неверный метод сэмплирования'
    
    target_counts = df[target_name].value_counts()

    major_class_name = target_counts.argmax()
    minor_class_name = target_counts.argmin()

    disbalance_coeff = int(target_counts[major_class_name] / target_counts[minor_class_name]) - 1
    if method == 'over':
        for i in range(disbalance_coeff):
            sample = df[df[target_name] == minor_class_name].sample(target_counts[minor_class_name])
            df = df.append(sample, ignore_index=True)
            
    elif method == 'under':
        df_ = df.copy()
        df = df_[df_[target_name] == minor_class_name]
        tmp = df_[df_[target_name] == major_class_name]
        df = df.append(tmp.iloc[
            np.random.randint(0, tmp.shape[0], target_counts[minor_class_name])
        ], ignore_index=True)

    elif method == 'tomek':
        from imblearn.under_sampling import TomekLinks
        tl = TomekLinks()
        X_tomek, y_tomek = tl.fit_sample(df.drop(columns=target_name), df[target_name])
        df = pd.concat([X_tomek, y_tomek], axis=1)
    
    elif method == 'smote':
        from imblearn.over_sampling import SMOTE
        smote = SMOTE()
        X_smote, y_smote = smote.fit_sample(df.drop(columns=target_name), df[target_name])
        df = pd.concat([X_smote, y_smote], axis=1)

    return df.sample(frac=1) 

In [182]:
def evalute_model(df, estimator: object, metric: callable, is_balanced=True):

 # Разделим выборки на тренировчную, тестовую и валидационную.
    x_train, x_test = train_test_split(df.drop(["APPLICATION_NUMBER", "TARGET"], axis=1), train_size=0.60, shuffle=False, random_state=1)
    y_train, y_test = train_test_split(df["TARGET"], train_size=0.60, shuffle=False, random_state=1,) # В данных имеется временная зависимость, поэтому пока перемешивать ничего не будем.

    x_test, x_val = train_test_split(x_test, train_size=0.50, shuffle=False, random_state=1,)
    y_test, y_val = train_test_split(y_test, train_size=0.50, shuffle=False, random_state=1,)

    if is_balanced:
      df_for_balancing = pd.concat([x_train, y_train], axis=1)
      df_balanced = balance_df_by_target(df_for_balancing, "TARGET", method='over')
          
      x_train = df_balanced.drop(columns="TARGET")
      y_train = df_balanced["TARGET"]

    estimator = estimator
    
    estimator.fit(
        X=x_train,
        y=y_train,
        eval_set=[(x_train, y_train), (x_val, y_val)],
        verbose=20
    )

    train_score =  metric(y_train, estimator.predict_proba(x_train)[:,1])
    val_score = metric(y_val, estimator.predict_proba(x_val)[:,1])
    test_score = metric(y_test, estimator.predict_proba(x_test)[:,1])
   
    print(f"\r\n Train-score: {round(train_score, 3)}, Val-score: {round(val_score, 3)}, Test-score: {round(test_score, 3)}")

    return estimator

In [183]:
lgbm_model = lgb.LGBMClassifier(
        boosting_type="gbdt",
        objective ="binary",
        metric = "auc",
        learning_rate = 0.1,
        n_estimators = 10000,
        reg_lambda = 50,
        max_depth = 4,
        random_state = 1,
        early_stopping_round=50
       )

In [184]:
catb_model = catb.CatBoostClassifier(silent=True, random_state=21,
                                    score_function='L2',
                                    depth = 7,
                                    eval_metric='AUC',
                                    use_best_model=True,
                                    boosting_type = 'Ordered', #  / Plain 
                                    l2_leaf_reg = 3, # -- регуляризация. 3.0 = по умолчанию. 
                                    # eta = 0.5,
                                    learning_rate=0.01,
                                    custom_metric=['Precision', 'Recall'],
                                    iterations=10000,
                                    early_stopping_rounds=50,
                                    # verbose=True
                                )

In [185]:
df = df_prepare(train_df)

In [186]:
df.head()

APPLICATION_NUMBER  TARGET  NAME_CONTRACT_TYPE_Credit_Card  \
0           123687442       0                               0   
1           123597908       1                               0   
2           123526683       0                               0   
3           123710391       1                               0   
4           123590329       1                               0   

   NAME_CONTRACT_TYPE_Cash  GENDER  CHILDRENS  FAMILY_STATUS_Widow  \
0                        1       1        1.0                  0.0   
1                        1       0        NaN                  NaN   
2                        1       0        0.0                  0.0   
3                        1       1        0.0                  0.0   
4                        1       0        NaN                  NaN   

   FAMILY_STATUS_Unknown  FAMILY_STATUS_Single_/_not_married  \
0                    0.0                                 0.0   
1                    NaN                                 NaN   
2                    0.0                                 0.0   
3                    0.0                                 0.0   
4                    NaN                                 NaN   

   FAMILY_STATUS_Separated  ...  AMT_REQ_CREDIT_BUREAU_MON  \
0                      0.0  ...                        0.0   
1                      NaN  ...                        NaN   
2                      0.0  ...                        7.0   
3                      0.0  ...                        0.0   
4                      NaN  ...                        NaN   

   AMT_REQ_CREDIT_BUREAU_QRT  AMT_REQ_CREDIT_BUREAU_YEAR  active_credits  \
0                        0.0                         2.0             0.0   
1                        NaN                         NaN             4.0   
2                        0.0                         4.0             2.0   
3                        0.0                         0.0             0.0   
4                        NaN                         NaN             1.0   

   closed_credits  sold_credits  bad_debt  credit_problems  credits_count  \
0             1.0           0.0       0.0              0.0            1.0   
1             5.0           0.0       0.0              0.0            9.0   
2             7.0           0.0       0.0              0.0            9.0   
3             1.0           0.0       0.0              0.0            1.0   
4             2.0           0.0       0.0              0.0            3.0   

   curency_mean  
0           0.0  
1           0.0  
2           0.0  
3           0.0  
4           0.0  

[5 rows x 43 columns]

In [207]:
lgbm = evalute_model(df, lgbm_model, roc_auc_score, is_balanced=False)

[20]	training's auc: 0.715903	valid_1's auc: 0.712349
[40]	training's auc: 0.726847	valid_1's auc: 0.718625
[60]	training's auc: 0.734089	valid_1's auc: 0.721518
[80]	training's auc: 0.739398	valid_1's auc: 0.72238
[100]	training's auc: 0.744126	valid_1's auc: 0.722473
[120]	training's auc: 0.748314	valid_1's auc: 0.722672
[140]	training's auc: 0.752036	valid_1's auc: 0.723428
[160]	training's auc: 0.754812	valid_1's auc: 0.723168
[180]	training's auc: 0.757849	valid_1's auc: 0.723883
[200]	training's auc: 0.760212	valid_1's auc: 0.723809
[220]	training's auc: 0.762821	valid_1's auc: 0.723803
[240]	training's auc: 0.76515	valid_1's auc: 0.724078
[260]	training's auc: 0.768695	valid_1's auc: 0.724038
[280]	training's auc: 0.770522	valid_1's auc: 0.72412
[300]	training's auc: 0.772735	valid_1's auc: 0.723924

 Train-score: 0.767, Val-score: 0.725, Test-score: 0.715


In [208]:
lgbm_balanced = evalute_model(df, lgbm_model, roc_auc_score, is_balanced=True)

[20]	training's auc: 0.721795	valid_1's auc: 0.715316
[40]	training's auc: 0.738136	valid_1's auc: 0.721337
[60]	training's auc: 0.748406	valid_1's auc: 0.722059
[80]	training's auc: 0.75474	valid_1's auc: 0.723398
[100]	training's auc: 0.759756	valid_1's auc: 0.724196
[120]	training's auc: 0.766301	valid_1's auc: 0.724571
[140]	training's auc: 0.770937	valid_1's auc: 0.723858
[160]	training's auc: 0.775251	valid_1's auc: 0.723736

 Train-score: 0.766, Val-score: 0.725, Test-score: 0.711


In [69]:
# Оптимизируем данные. обернем все в pipline. 

In [69]:
# Подготовка данных для датасета.

In [209]:
catboost = evalute_model(df, catb_model, roc_auc_score, is_balanced=False)

0:	test: 0.6623500	test1: 0.6689050	best: 0.6689050 (0)	total: 152ms	remaining: 25m 17s
20:	test: 0.6882128	test1: 0.6972094	best: 0.6972094 (20)	total: 3.25s	remaining: 25m 43s
40:	test: 0.6982616	test1: 0.7049422	best: 0.7049422 (40)	total: 7.39s	remaining: 29m 54s
60:	test: 0.7043126	test1: 0.7104528	best: 0.7104528 (60)	total: 11.7s	remaining: 31m 42s
80:	test: 0.7097899	test1: 0.7144762	best: 0.7144762 (80)	total: 16.3s	remaining: 33m 11s
100:	test: 0.7138832	test1: 0.7172237	best: 0.7172911 (99)	total: 20.6s	remaining: 33m 43s
120:	test: 0.7165605	test1: 0.7190503	best: 0.7190503 (120)	total: 25.1s	remaining: 34m 6s
140:	test: 0.7196253	test1: 0.7203427	best: 0.7203427 (140)	total: 29.5s	remaining: 34m 22s
160:	test: 0.7220735	test1: 0.7216986	best: 0.7216986 (160)	total: 33.9s	remaining: 34m 31s
180:	test: 0.7238029	test1: 0.7220950	best: 0.7221454 (175)	total: 36.9s	remaining: 33m 20s
200:	test: 0.7257010	test1: 0.7226320	best: 0.7227184 (198)	total: 39.5s	remaining: 32m 7s
220

In [210]:
catboost_balanced = evalute_model(df, catb_model, roc_auc_score, is_balanced=True)

0:	test: 0.6862876	test1: 0.6912442	best: 0.6912442 (0)	total: 221ms	remaining: 36m 52s
20:	test: 0.7082248	test1: 0.7100512	best: 0.7100512 (20)	total: 7.51s	remaining: 59m 28s
40:	test: 0.7141340	test1: 0.7124458	best: 0.7124458 (40)	total: 11.4s	remaining: 46m 2s
60:	test: 0.7187804	test1: 0.7145348	best: 0.7145673 (59)	total: 14.9s	remaining: 40m 31s
80:	test: 0.7233921	test1: 0.7174094	best: 0.7174094 (80)	total: 17.6s	remaining: 35m 57s
100:	test: 0.7294174	test1: 0.7196104	best: 0.7197386 (99)	total: 20.9s	remaining: 34m 11s
120:	test: 0.7340069	test1: 0.7211448	best: 0.7211448 (120)	total: 27.1s	remaining: 36m 53s
140:	test: 0.7377230	test1: 0.7213094	best: 0.7213377 (139)	total: 34.3s	remaining: 40m
160:	test: 0.7412838	test1: 0.7221308	best: 0.7222016 (159)	total: 43.9s	remaining: 44m 42s
180:	test: 0.7445196	test1: 0.7222684	best: 0.7224448 (171)	total: 55.6s	remaining: 50m 14s
200:	test: 0.7478179	test1: 0.7220303	best: 0.7224448 (171)	total: 1m 5s	remaining: 53m 11s
220:	t

In [190]:
# Разделим выборки на тренировчную, тестовую и валидационную.
x_train, x_test = train_test_split(df.drop(["APPLICATION_NUMBER", "TARGET"], axis=1), train_size=0.80, shuffle=False, random_state=1)
y_train, y_test = train_test_split(df["TARGET"], train_size=0.80, shuffle=False, random_state=1,) # В данных имеется временная зависимость, поэтому пока перемешивать ничего не будем.

df_for_balancing = pd.concat([x_train, y_train], axis=1)
df_balanced = balance_df_by_target(df_for_balancing, "TARGET", method='over')

x_train_balanced = df_balanced.drop(["TARGET"], axis=1)
y_train_balanced = df_balanced["TARGET"]

In [193]:
x_train.shape

(88074, 41)

In [194]:
x_train_balanced.shape

(160194, 41)

In [195]:
def make_cross_validation(X: pd.DataFrame,
                          y: pd.Series,
                          estimator: object,
                          metric: callable,
                          cv_strategy):
    """
    Кросс-валидация.

    Parameters
    ----------
    X: pd.DataFrame
        Матрица признаков.

    y: pd.Series
        Вектор целевой переменной.

    estimator: callable
        Объект модели для обучения.

    metric: callable
        Метрика для оценки качества решения.
        Ожидается, что на вход будет передана функция,
        которая принимает 2 аргумента: y_true, y_pred.

    cv_strategy: cross-validation generator
        Объект для описания стратегии кросс-валидации.
        Ожидается, что на вход будет передан объект типа
        KFold или StratifiedKFold.

    Returns
    -------
    oof_score: float
        Значение метрики качества на OOF-прогнозах.

    fold_train_scores: List[float]
        Значение метрики качества на каждом обучающем датасете кросс-валидации.

    fold_valid_scores: List[float]
        Значение метрики качества на каждом валидационном датасете кросс-валидации.

    oof_predictions: np.array
        Прогнозы на OOF.

    """
    estimators, fold_train_scores, fold_valid_scores = [], [], []
    oof_predictions = np.zeros(X.shape[0])

    for fold_number, (train_idx, valid_idx) in enumerate(cv_strategy.split(X, y)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]
      
        estimator.fit(x_train, y_train,
                      eval_set=[(x_train, y_train), (x_valid, y_valid)],
                      verbose=False
                      )
        y_train_pred = estimator.predict_proba(x_train)[:,1]
        y_valid_pred = estimator.predict_proba(x_valid)[:,1]
        # print(y_train_pred.shape, y_train.shape)
        fold_train_scores.append(metric(y_train, y_train_pred))
        fold_valid_scores.append(metric(y_valid, y_valid_pred))
        oof_predictions[valid_idx] = y_valid_pred

        msg = (
            f"Fold: {fold_number+1}, train-observations = {len(train_idx)}, "
            f"valid-observations = {len(valid_idx)}\n"
            f"train-score = {round(fold_train_scores[fold_number], 4)}, "
            f"valid-score = {round(fold_valid_scores[fold_number], 4)}" 
        )
        print(msg)
        print("="*69)
        estimators.append(estimator)

    oof_score = metric(y, oof_predictions)
    print(f"CV-results train: {round(np.mean(fold_train_scores), 4)} +/- {round(np.std(fold_train_scores), 3)}")
    print(f"CV-results valid: {round(np.mean(fold_valid_scores), 4)} +/- {round(np.std(fold_valid_scores), 3)}")
    print(f"OOF-score = {round(oof_score, 4)}")

    return estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions

In [77]:
cv_strategy = KFold(n_splits=5, random_state=1, shuffle=True)

lbgm_estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
   x_train, y_train, lgbm_model, metric=roc_auc_score, cv_strategy=cv_strategy )

Fold: 1, train-observations = 95993, valid-observations = 23999
train-score = 0.7626, valid-score = 0.728
Fold: 2, train-observations = 95993, valid-observations = 23999
train-score = 0.7884, valid-score = 0.7275
Fold: 3, train-observations = 95994, valid-observations = 23998
train-score = 0.7624, valid-score = 0.7353
Fold: 4, train-observations = 95994, valid-observations = 23998
train-score = 0.7776, valid-score = 0.7185
Fold: 5, train-observations = 95994, valid-observations = 23998
train-score = 0.7769, valid-score = 0.732
CV-results train: 0.7736 +/- 0.01
CV-results valid: 0.7283 +/- 0.006
OOF-score = 0.7279


In [118]:
cv_strategy = StratifiedKFold(n_splits=5, random_state=1, shuffle=True)

lgbm_estimators_stratify, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
   x_train, y_train, lgbm_model, metric=roc_auc_score, cv_strategy=cv_strategy
)

Fold: 1, train-observations = 95993, valid-observations = 23999
train-score = 0.7752, valid-score = 0.7309
Fold: 2, train-observations = 95993, valid-observations = 23999
train-score = 0.7735, valid-score = 0.7211
Fold: 3, train-observations = 95994, valid-observations = 23998
train-score = 0.7705, valid-score = 0.7284
Fold: 4, train-observations = 95994, valid-observations = 23998
train-score = 0.7811, valid-score = 0.7248
Fold: 5, train-observations = 95994, valid-observations = 23998
train-score = 0.7815, valid-score = 0.7326
CV-results train: 0.7764 +/- 0.004
CV-results valid: 0.7276 +/- 0.004
OOF-score = 0.7274


In [78]:
cv_strategy = KFold(n_splits=5, random_state=1, shuffle=True)

catb_estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
   x_train, y_train, catb_model, metric=roc_auc_score, cv_strategy=cv_strategy )

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

Fold: 1, train-observations = 95993, valid-observations = 23999
train-score = 0.7681, valid-score = 0.7297
Fold: 2, train-observations = 95993, valid-observations = 23999
train-score = 0.797, valid-score = 0.7291
Fold: 3, train-observations = 95994, valid-observations = 23998
train-score = 0.7733, valid-score = 0.7395
Fold: 4, train-observations = 95994, valid-observations = 23998
train-score = 0.7909, valid-score = 0.7222
Fold: 5, train-observations = 95994, valid-observations = 23998
train-score = 0.786, valid-score = 0.7379
CV-results train: 0.7831 +/- 0.011
CV-results valid: 0.7317 +/- 0.006
OOF-score = 0.7312


In [196]:
cv_strategy = StratifiedKFold(n_splits=5, random_state=1, shuffle=True)

catb_estimators_stratify, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
   x_train, y_train, catb_model, metric=roc_auc_score, cv_strategy=cv_strategy
)

Fold: 1, train-observations = 70459, valid-observations = 17615
train-score = 0.7662, valid-score = 0.7235
Fold: 2, train-observations = 70459, valid-observations = 17615
train-score = 0.7396, valid-score = 0.7148
Fold: 3, train-observations = 70459, valid-observations = 17615
train-score = 0.743, valid-score = 0.7261
Fold: 4, train-observations = 70459, valid-observations = 17615
train-score = 0.7507, valid-score = 0.714
Fold: 5, train-observations = 70460, valid-observations = 17614
train-score = 0.7651, valid-score = 0.7149
CV-results train: 0.7529 +/- 0.011
CV-results valid: 0.7187 +/- 0.005
OOF-score = 0.7181


In [219]:
## Прогноз

y_pred = np.zeros(x_test.shape[0])

for estimator in catb_estimators_stratify:
    y_pred += estimator.predict_proba(x_test)[:, 1]

y_pred = y_pred/cv_strategy.n_splits

score = roc_auc_score(y_test, y_pred)
print(f'Test predict = {score:.5}')

Test predict = 0.72271


In [ ]:
# Обработать целевой датасет. вычислить результат. Загрузить на платформу.

In [198]:
test_df = pd.read_csv('../data/geekbrains-competitive-data-analysis/test.csv')
test_df.head(4)

APPLICATION_NUMBER NAME_CONTRACT_TYPE
0           123724268               Cash
1           123456549               Cash
2           123428178        Credit Card
3           123619984               Cash

In [221]:
test = df_prepare(test_df)

final_id = test['APPLICATION_NUMBER']
x_data = test.drop(["APPLICATION_NUMBER"], axis=1)

In [222]:
# прогноз
y_pred = np.zeros(x_data.shape[0])

for estimator in catb_estimators_stratify:
    y_pred += estimator.predict_proba(x_data)[:, 1]
    
y_pred=y_pred/cv_strategy.n_splits

In [223]:
y_true = np.zeros(x_data.shape[0])

In [224]:
df_final = pd.DataFrame({'APPLICATION_NUMBER':final_id, 'TARGET':list(y_pred)})

In [225]:
df_final.head(5)

APPLICATION_NUMBER    TARGET
0           123724268  0.514603
1           123456549  0.664110
2           123428178  0.555442
3           123619984  0.512657
4           123671104  0.347828

In [226]:
df_final.to_csv('prediction_best.csv', index=False, encoding='utf-8')

In [227]:
df_final = pd.read_csv('prediction_best.csv')

In [228]:
df_final.shape

(165141, 2)